In [13]:
import telebot
import pandas as pd

from gtts import gTTS


import pickle
from PIL import Image
import numpy as np

from sklearn.preprocessing import LabelEncoder
import torch
from torchvision import models
from torchvision import transforms


In [14]:
label_encoder = pickle.load(open("../label_encoder.pkl", 'rb'))

cnn = models.densenet201(num_classes=42).to(torch.device('cpu'))
cnn.load_state_dict(torch.load('../model/model_densnet201'))


RESCALE_SIZE = 128

def preprocess (image):
        # для преобразования изображений в тензоры PyTorch и нормализации входа
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
        ])
        x = image
        x = np.array(image.resize((RESCALE_SIZE, RESCALE_SIZE)))
        x = np.array(x / 255, dtype='float32')
        x = transform(x)
        return x

def predict_one_sample(model, image):
    """Предсказание, для одной картинки"""
    #image_tensor = preprocess(image)
    with torch.no_grad():
        #inputs = inputs.to(device)
        model.eval()
        logit = model(image).cpu()
        probs = torch.nn.functional.softmax(logit, dim=-1).numpy()
    return probs

In [15]:
class_normal_name = {v:' '.join(v.split('_')) for v in label_encoder.classes_}

In [16]:
class_normal_name ={
    'abraham_grampa_simpson': 'abraham grampa simpson',
    'agnes_skinner': 'agnes skinner',
    'apu_nahasapeemapetilon': 'apu nahasapeemapetilon',
    'barney_gumble': 'barney gumble',
    'bart_simpson': 'bart simpson',
    'carl_carlson': 'carl carlson',
    'charles_montgomery_burns': 'charles montgomery burns',
    'chief_wiggum': 'chief wiggum',
    'cletus_spuckler': 'cletus spuckler',
    'comic_book_guy': 'comic book guy',
    'disco_stu': 'disco stu',
    'edna_krabappel': 'edna krabappel',
    'fat_tony': 'fat tony',
    'gil': 'gil',
    'groundskeeper_willie': 'groundskeeper willie',
    'homer_simpson': 'homer simpson',
    'kent_brockman': 'kent brockman',
    'krusty_the_clown': 'krusty the clown',
    'lenny_leonard': 'lenny leonard',
    'lionel_hutz': 'lionel hutz',
    'lisa_simpson': 'lisa simpson',
    'maggie_simpson': 'maggie simpson',
    'marge_simpson': 'marge simpson',
    'martin_prince': 'martin prince',
    'mayor_quimby': 'mayor quimby',
    'milhouse_van_houten': 'milhouse van houten',
    'miss_hoover': 'miss hoover',
    'moe_szyslak': 'moe szyslak',
    'ned_flanders': 'ned flanders',
    'nelson_muntz': 'nelson muntz',
    'otto_mann': 'otto mann',
    'patty_bouvier': 'patty bouvier',
    'principal_skinner': 'principal skinner',
    'professor_john_frink': 'professor john frink',
    'rainier_wolfcastle': 'rainier wolfcastle',
    'ralph_wiggum': 'ralph wiggum',
    'selma_bouvier': 'selma bouvier',
    'sideshow_bob': 'sideshow bob',
    'sideshow_mel': 'sideshow mel',
    'snake_jailbird': 'snake jailbird',
    'troy_mcclure': 'troy mcclure',
    'waylon_smithers': 'waylon smithers'
}

In [17]:
def get_submit():
    image_tesnor = preprocess(Image.open('./image/image.jpg')).reshape(((1, 3, 128, 128)))
    predicted = predict_one_sample(cnn, image_tesnor).argmax()
    return label_encoder.classes_[predicted]


In [18]:
def creat_ru_audio(text):
    tts = gTTS(text, lang='ru', slow=False)
    tts.save('sound_ru.mp3')
    
def creat_en_audio(text):
    tts = gTTS(text, lang='en', slow=False)
    tts.save('sound_en.mp3')

In [19]:

#Сюда подставляете свой токен
bot = telebot.TeleBot('5358515008:AAGVmnGBgtnr7Prnm48VwVyeE9bniigr_Fk')

@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Пришли фото персонажа из симпсонов и я его распознаю.')


@bot.message_handler(content_types=['photo'])
def handle_docs_photo(message):
    file_info = bot.get_file(message.photo[-1].file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    with open('./image/image.jpg', 'wb') as new_file:
        new_file.write(downloaded_file)
    
    answer = get_submit()
    answer = class_normal_name[answer]
    creat_en_audio(answer)
    with open('sound_en.mp3', 'rb') as audio:
        bot.send_voice(message.chat.id, audio)
    

@bot.message_handler(content_types=['text'])
def send_text(message='text'):
    bot.send_message(message.chat.id, 'Пришли фото персонажа из симпсонов и я его распознаю.')
    #creat_ru_audio(message.text)
    #with open('sound_ru.mp3', 'rb') as audio:
    #    bot.send_voice(message.chat.id, audio)

    

bot.polling()
#сами смайлики прописать не могу, иначе публикация не обрабатывается на habr